# Data pre-processing
1. Using selective search to find proposed objects from each image
2. Using IoU to check if proposed object box overlaps with any of the real boxes and if they do, we mark them as positives samples, and if not, we mark them as negative samples.

In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from utils.IoU import compute_iou

### Setting up paths here for original images and labels path

In [2]:
cwd = os.getcwd()

original_data_path = "original_data"
original_images_path = os.path.join(cwd, original_data_path, "images")
original_labels_path = os.path.join(cwd, original_data_path, "labels")

### Setting up paths here for proposed images and labels path

In [3]:
base_path = "proposed_dataset"
proposed_car = os.path.join(cwd, base_path, "car")
proposed_non_car = os.path.join(cwd, base_path, "non_car")

### Max proposed objects per image found using selective search

In [4]:
max_proposals = 2000
max_proposal_infer = 200

### Max number of positive and negative samples to be generated per image

In [5]:
max_positive = 30
max_negative = 10

In [6]:
input_dimension = (224, 224)
model_path = "car_detector.h5"

In [7]:
def selective_search(image, method="fast"):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)
    if method == "fast":
        ss.switchToSelectiveSearchFast()
    else:
        ss.switchToSelectiveSearchQuality()
    rects = ss.process()
    return rects

In [8]:
def load_image(image_path):
    """
    Load image from path and convert it to RGB and resize it to (224, 224)
    Parameters:
        image_path: path to image
    Returns:
        image: ndarray of shape (224, 224, 3)
    """
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, input_dimension)
    return image

In [9]:
def load_label(label_path):
    """
    Loads all the label information from the label file for one image. Makes dictionary with all the objects found in the file.
    Parameters:
        label_path: path to label
    Returns:
        label: dictionary
    """
    with open(label_path, "r") as f:
        lines = f.readlines()
    labels = {}
    for line in lines:
        line = line.strip().split(" ")
        labels[line[0]] = {
            "x": int(float(line[1]) * input_dimension[0]),
            "y": int(float(line[2]) * input_dimension[1]),
            "w": int(float(line[3]) * input_dimension[0]),
            "h": int(float(line[4]) * input_dimension[1]),
        }
    return labels

In [10]:
def save_image(image, image_path):
    """
    Saves image to path
    Parameters:
        image: ndarray of shape (224, 224, 3)
        image_path: path to save image
    """
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(image_path, image)

In [11]:
def check_all_rectangles(rect, labels):
    """
    Checks if proposed object box overlaps with any of the real boxes and if they do, we mark them as positives samples, and if not, we mark them as negative samples.
    Parameters:
        rect: proposed object box
        labels: dictionary of real boxes
    Returns:
        label: 1 if positive sample, 0 if negative sample
    """
    print(labels)
    print(rect)

In [12]:
images_to_process = len(os.listdir(original_images_path))
print("Total images to process: ", images_to_process)

Total images to process:  46


In [14]:
for i, image_name in enumerate(os.listdir(original_images_path)[:1]):
    print("Processing image: ", i + 1, "/", images_to_process)
    image_path = os.path.join(original_images_path, image_name)
    label_path = os.path.join(original_labels_path, image_name.split(".")[0] + ".txt")
    image = load_image(image_path)
    labels = load_label(label_path)
    rects = selective_search(image)
    positive_count = 0
    negative_count = 0

    for rect in rects[:max_proposals]:
        x, y, w, h = rect
        if x < 0 or y < 0 or w < 0 or h < 0:
            continue
        if x + w > image.shape[1] or y + h > image.shape[0]:
            continue
        check_all_rectangles(rect, labels) # TODO check if this is correct

Processing image:  1 / 46
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[  0 204  28  17]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[ 59   6 165   1]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[158  43  66  98]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[ 60   0 164   5]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[ 32  85 192 125]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[ 46   2 178   5]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[136 129  21  14]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[ 59   2 165   4]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[134 128  23   9]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[  0  59 133  51]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[  0   7 224   2]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[152   9  72  34]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[73 30 68 33]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[ 59   0 165   7]
{'0': {'x': 174, 'y': 97, 'w': 18, 'h': 13}}
[220  83   4  34]
{'0': {'x': 174, 'y': 97, 'w': 18